# Examples #

In [ ]:
# imports
import os;
import sys;

# NOTE: need this to force jupyter to reload imports:
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key];

os.chdir(os.path.dirname(_dh[0]));
sys.path.insert(0, os.getcwd());

from src.thirdparty.maths import *;
from src.thirdparty.quantum import *;
from src.thirdparty.render import *;

from src.api.ibm import *;
from src.algorithms import *;

np.random.seed(39102901); # for repeatability

In [ ]:
# NOTE: Avoid repeating this cell.
provider = connect_to_ibm_account(force_reload=False);
display(provider.backends())

In [ ]:
# Simulation with Aer backend
with CreateBackend(
    provider = provider,
    kind = BACKEND_SIMULATOR.AER,
) as backend:
    circuit = QuantumCircuit(2);
    circuit.h(0);
    circuit.cx(0, 1);
    circuit.measure_all();
    display(circuit.draw(initial_state=True));
    job = backend.run(circuit);
    result = job.result();
    counts = result.get_counts();
    display(QkVisualisation.plot_histogram(counts));

In [ ]:
with CreateBackend(
    provider = provider,
    kind = BACKEND_SIMULATOR.UNITARY,
) as backend:
    circuit = QuantumCircuit(1);
    circuit.u(np.pi/2,np.pi/2,np.pi/2,0);
    display(circuit.draw(initial_state=False));
    job = backend.run(QkTranspile(circuit, backend));
    result = job.result();
    display(array_to_latex(result.get_unitary(circuit, decimals=3)));